In [ ]:
import numpy as np
from src import utils, hiwa
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.manifold import Isomap
import time
import matplotlib.pyplot as plt

In [ ]:
T_te, T_tr, X_te, X_tr, Y_te, Y_tr = utils.load_data('data\\mihi_demo.mat')

Y_te_3d = FactorAnalysis(n_components=3).fit_transform(utils.remove_const_cols(Y_te))
X_tr_3d = utils.map_X_3D(X_tr)
X_transform = np.linalg.pinv(X_tr_3d) @ X_tr

t1 = time.time()
hwa = hiwa.HiWA(dim_red_method=Isomap(n_components=2), normal=True)
hwa.fit(Y_te_3d, T_te.squeeze(), X_tr_3d, T_tr.squeeze(), Y_transform=X_transform, Rgt=utils.LS_oracle(utils.map_X_3D(X_te), Y_te_3d))
Y_te_rec = hwa.transform(Y_te_3d)
t2 = time.time()

r2_hwa = utils.eval_R2(Y_te_rec, utils.map_X_3D(X_te))
print(t2 - t1)

In [ ]:
plt.figure(figsize=(20, 6))
plt.subplot(1, 5, 1)
plt.title('Training')
utils.color_data(X_tr, T_tr)
plt.subplot(1, 5, 2)
plt.title('HiWA ($R^2 = $ %.4f)' %r2_hwa)
utils.color_data(Y_te_rec, T_te)
ax = plt.subplot(1, 5, 3)
plt.plot(hwa.diagnostics['Rg_norm'])
ax.set_yscale('log')
plt.subplot(1, 5, 4)
plt.imshow(hwa.P)
plt.xticks(np.arange(4))
plt.yticks(np.arange(4))
plt.title('Entropy$ = $ %.4f' %(-np.sum(hwa.P * np.log(hwa.P))))
plt.subplot(1, 5, 5)
plt.imshow(hwa.diagnostics['C'])
c = plt.colorbar()